In [17]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import pickle
import tensorflow as tf
from tensorflow import keras
from keras.models import load_model

In [18]:
## load the trained model : model.h5, load the scaler & encoder file:
model = load_model('model.h5')

with open('label_encoder_gender.pkl','rb') as file:
    label_encoder_gender = pickle.load(file)
    
with open('onehot_encoder_geo.pkl', 'rb') as file:
    onehot_encoder = pickle.load(file)
    
with open('scaler.pkl', 'rb') as file:
    scaler = pickle.load(file)

In [19]:
## example input data
input_data = {
    'CreditScore': 600,
    'Geography': 'France',
    'Gender': 'Male',
    'Age': 40,
    'Tenure': 5,
    'Balance': 100000,
    'NumOfProducts': 3,
    'HasCrCard': 1,
    'IsActiveMember': 1,
    'EstimatedSalary': 50000
}

In [20]:
## one-hot encoded 'Geography'
geo_encoded = onehot_encoder.transform([[input_data['Geography']]]).toarray()
geo_encoded_df = pd.DataFrame(geo_encoded, columns=onehot_encoder.get_feature_names_out(['Geography']))

c:\Users\dell\Desktop\Projects\Client-Retention-Insight\venv\Lib\site-packages\sklearn\utils\validation.py:2739: UserWarning: X does not have valid feature names, but OneHotEncoder was fitted with feature names
  warnings.warn(


In [21]:
geo_encoded_df

,Geography_France,Geography_Germany,Geography_Spain
0,1.0,0.0,0.0


In [22]:
input_df = pd.DataFrame([input_data])
input_df

,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary
0,600,France,Male,40,5,100000,3,1,1,50000


In [23]:
## Encode categorical variables
input_df['Gender'] = label_encoder_gender.transform(input_df['Gender'])
input_df

,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary
0,600,France,1,40,5,100000,3,1,1,50000


In [24]:
## cocatination of the input data
import pandas as pd
input_df = pd.concat([input_df.drop("Geography", axis=1), geo_encoded_df], axis=1)

In [25]:
input_df

,CreditScore,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Geography_France,Geography_Germany,Geography_Spain
0,600,1,40,5,100000,3,1,1,50000,1.0,0.0,0.0


In [28]:
## scaled the input_data
input_scaled = scaler.transform(input_df)
input_scaled

array([[-5.35985161e-01,  9.13247552e-01,  1.04793593e-01,
        -1.34471949e-03,  3.82628391e-01,  2.53355998e+00,
         6.49202671e-01,  9.74816989e-01, -8.76832208e-01,
         1.00150113e+00, -5.79467227e-01, -5.76388018e-01]])

In [29]:
## Predict the churn : customer retention
prediction = model.predict(input_scaled)
prediction

1/1 [==============================] - 0s 255ms/step


array([[0.84067005]], dtype=float32)

In [30]:
## probability of churn:
prediction_prob = prediction[0][0]

if prediction_prob > 0.5:
    print('customer is likely to churn')
else:
    print('customer is not likely to churn')

customer is likely to churn
